# 9 WorkFlow Analista Jr

### 9.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán
<br>El Analista Jr corre sus scripts en la virtual manchine **desktop-jr** que tiene estas características


*   Normal, paga tarifa completa, nunca es apagada por Google
*   reside en el datacenter de Sao Paulo, Brasil
*   64 GB de memoria RAM
*   8 vCPU



## 9.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Aug 24 16:57:48 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656844,35.1,1439320,76.9,1439320,76.9
Vcells,1224851,9.4,8388608,64.0,1924961,14.7


In [3]:
# --- Configuración ntfy ---
Sys.setenv(NTFY_SERVER = "https://ntfy.sh")
Sys.setenv(NTFY_CHANNEL = "Juanmg1984")   # tu canal

# --- Helper para enviar notificaciones ---
ntfy_send <- function(message, title = "Notebook", priority = 3, tags = NULL) {
  channel <- Sys.getenv("NTFY_CHANNEL")
  server  <- Sys.getenv("NTFY_SERVER", "https://ntfy.sh")
  stopifnot(nzchar(channel))
  url <- sprintf("%s/%s", sub("/$", "", server), channel)

  hdr <- c(sprintf("Title: %s", title), sprintf("Priority: %s", priority))
  if (!is.null(tags)) hdr <- c(hdr, sprintf("Tags: %s", paste(tags, collapse=",")))

  args <- c(rbind("-H", shQuote(hdr)), "-d", shQuote(message), shQuote(url))
  invisible(system2("curl", args = args))
}

# Azúcar
notificar <- function(m) ntfy_send(m, title = "Notebook", priority = 3, tags = c("bell"))

# Wrapper para bloques pesados
notificar_chunk <- function(nombre, expr) {
  inicio <- Sys.time()
  ntfy_send(sprintf("▶️ Comienza: %s", nombre), priority = 4, tags = c("play","hourglass"))
  on.exit({
    fin <- Sys.time()
    ntfy_send(sprintf("✅ Termina: %s (%.1f min)", nombre,
                      as.numeric(difftime(fin, inicio, units="mins"))),
              priority = 3, tags = c("checkered_flag"))
  }, add = TRUE)
  force(expr)
}


In [4]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




#### Parametros

In [5]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191

PARAM$experimento <- 1236
PARAM$dataset <- "analistajr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [6]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 9.3.1   Preprocesamiento del dataset

#### 9.3.1.1  DT incorporar dataset

In [7]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 9.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [8]:
if( !require("mice")) install.packages("mice", repos = "http://cran.us.r-project.org")
require("mice")

Loading required package: mice


Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind




In [9]:
# Escrito por alumnos de  Universidad Austral  Rosario

Corregir_MICE <- function(pcampo, pmeses) {

  meth <- rep("", ncol(dataset))
  names(meth) <- colnames(dataset)
  meth[names(meth) == pcampo] <- "sample"

  # llamada a mice  !
  imputacion <- mice(dataset,
    method = meth,
    maxit = 5,
    m = 1,
    seed = 7)

  tbl <- mice::complete(dataset)

  dataset[, paste0(pcampo) := ifelse(foto_mes %in% pmeses, tbl[, get(pcampo)], get(pcampo))]

}


In [10]:
Corregir_interpolar <- function(pcampo, pmeses) {

  tbl <- dataset[, list(
    "v1" = shift(get(pcampo), 1, type = "lag"),
    "v2" = shift(get(pcampo), 1, type = "lead")
  ),
  by = eval(envg$PARAM$dataset_metadata$entity_id)
  ]

  tbl[, paste0(envg$PARAM$dataset_metadata$entity_id) := NULL]
  tbl[, promedio := rowMeans(tbl, na.rm = TRUE)]

  dataset[
    ,
    paste0(pcampo) := ifelse(!(foto_mes %in% pmeses),
      get(pcampo),
      tbl$promedio
    )
  ]
}

In [11]:
AsignarNA_campomeses <- function(pcampo, pmeses) {

  if( pcampo %in% colnames( dataset ) ) {

    dataset[ foto_mes %in% pmeses, paste0(pcampo) := NA ]
  }
}

In [12]:

Corregir_atributo <- function(pcampo, pmeses, pmetodo)
{
  # si el campo no existe en el dataset, Afuera !
  if( !(pcampo %in% colnames( dataset )) )
    return( 1 )

  # llamo a la funcion especializada que corresponde
  switch( pmetodo,
    "MachineLearning"     = AsignarNA_campomeses(pcampo, pmeses),
    "EstadisticaClasica"  = Corregir_interpolar(pcampo, pmeses),
    "MICE"                = Corregir_MICE(pcampo, pmeses),
  )

  return( 0 )
}

In [13]:

Corregir_Rotas <- function(dataset, pmetodo) {
  gc(verbose= FALSE)
  cat( "inicio Corregir_Rotas()\n")
  # acomodo los errores del dataset

  Corregir_atributo("active_quarter", c(202006), pmetodo) # 1
  Corregir_atributo("internet", c(202006), pmetodo) # 2

  Corregir_atributo("mrentabilidad", c(201905, 201910, 202006), pmetodo) # 3
  Corregir_atributo("mrentabilidad_annual", c(201905, 201910, 202006), pmetodo) # 4

  Corregir_atributo("mcomisiones", c(201905, 201910, 202006), pmetodo) # 5

  Corregir_atributo("mactivos_margen", c(201905, 201910, 202006), pmetodo) # 6
  Corregir_atributo("mpasivos_margen", c(201905, 201910, 202006), pmetodo) # 7

  Corregir_atributo("mcuentas_saldo", c(202006), pmetodo) # 8

  Corregir_atributo("ctarjeta_debito_transacciones", c(202006), pmetodo) # 9

  Corregir_atributo("mautoservicio", c(202006), pmetodo) # 10

  Corregir_atributo("ctarjeta_visa_transacciones", c(202006), pmetodo) # 11
  Corregir_atributo("mtarjeta_visa_consumo", c(202006), pmetodo) # 12

  Corregir_atributo("ctarjeta_master_transacciones", c(202006), pmetodo) # 13
  Corregir_atributo("mtarjeta_master_consumo", c(202006), pmetodo) # 14

  Corregir_atributo("ctarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 15
  Corregir_atributo("mttarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 16

  Corregir_atributo("ccajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 17

  Corregir_atributo("mcajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 18

  Corregir_atributo("ctarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 19

  Corregir_atributo("mtarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 20

  Corregir_atributo("ctarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 21

  Corregir_atributo("mtarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 22

  Corregir_atributo("ccomisiones_otras", c(201905, 201910, 202006), pmetodo) # 23
  Corregir_atributo("mcomisiones_otras", c(201905, 201910, 202006), pmetodo) # 24

  Corregir_atributo("cextraccion_autoservicio", c(202006), pmetodo) # 25
  Corregir_atributo("mextraccion_autoservicio", c(202006), pmetodo) # 26

  Corregir_atributo("ccheques_depositados", c(202006), pmetodo) # 27
  Corregir_atributo("mcheques_depositados", c(202006), pmetodo) # 28
  Corregir_atributo("ccheques_emitidos", c(202006), pmetodo) # 29
  Corregir_atributo("mcheques_emitidos", c(202006), pmetodo) # 30
  Corregir_atributo("ccheques_depositados_rechazados", c(202006), pmetodo) # 31
  Corregir_atributo("mcheques_depositados_rechazados", c(202006), pmetodo) # 32
  Corregir_atributo("ccheques_emitidos_rechazados", c(202006), pmetodo) # 33
  Corregir_atributo("mcheques_emitidos_rechazados", c(202006), pmetodo) # 34

  Corregir_atributo("tcallcenter", c(202006), pmetodo) # 35
  Corregir_atributo("ccallcenter_transacciones", c(202006), pmetodo) # 36

  Corregir_atributo("thomebanking", c(202006), pmetodo) # 37
  Corregir_atributo("chomebanking_transacciones", c(201910, 202006), pmetodo) # 38

  Corregir_atributo("ccajas_transacciones", c(202006), pmetodo) # 39
  Corregir_atributo("ccajas_consultas", c(202006), pmetodo) # 40

  Corregir_atributo("ccajas_depositos", c(202006, 202105), pmetodo) # 41

  Corregir_atributo("ccajas_extracciones", c(202006), pmetodo) # 41
  Corregir_atributo("ccajas_otras", c(202006), pmetodo) # 43

  Corregir_atributo("catm_trx", c(202006), pmetodo) # 44
  Corregir_atributo("matm", c(202006), pmetodo) # 45
  Corregir_atributo("catm_trx_other", c(202006), pmetodo) # 46
  Corregir_atributo("matm_other", c(202006), pmetodo) # 47

  cat( "fin Corregir_rotas()\n")
}


In [14]:
# resuelvo el Catastrophe Analysis

setorder( dataset, numero_de_cliente, foto_mes )

PARAM$CA$metodo= "MachineLearning"

if( PARAM$CA$metodo %in% c("MachineLearning", "EstadisticaClasica", "MICE") )
  Corregir_Rotas(dataset, PARAM$CA$metodo)

inicio Corregir_Rotas()
fin Corregir_rotas()


#### 9.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, ajustando por algunos indices financieros

In [15]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)


In [16]:
# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.9903030878, 1.9174403544, 1.8296186587,
  1.7728862972, 1.7212488323, 1.6776304408,
  1.6431248196, 1.5814483345, 1.4947526791,
  1.4484037589, 1.3913580777, 1.3404220402,
  1.3154288912, 1.2921698342, 1.2472681797,
  1.2300475145, 1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219, 0.7566381305, 0.7289384687
)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.380952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vUVA <- c(
  2.001408838932958,  1.950325472789153,  1.89323032351521,
  1.8247220405493787, 1.746027787673673,  1.6871348409529485,
  1.6361678865622313, 1.5927529755859773, 1.5549162794128493,
  1.4949100586391746, 1.4197729500774545, 1.3678188186372326,
  1.3136508617223726, 1.2690535173062818, 1.2381595983200178,
  1.211656735577568,  1.1770808941405335, 1.1570338657445522,
  1.1388769475653255, 1.1156993751209352, 1.093638313080772,
  1.0657171590878205, 1.0362173587708712, 1.0,
  0.9669867858358365, 0.9323750098728378, 0.8958202912590305,
  0.8631993702994263, 0.8253893405524657, 0.7928918905364516,
  0.7666323845128089, 0.7428976357662823, 0.721615762047849
)


In [17]:
tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

IPC,dolar_blue,dolar_oficial,UVA,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.9903031,39.04545,38.43000,2.0014088,201901
1.9174404,38.40250,39.42800,1.9503255,201902
1.8296187,41.63947,42.54210,1.8932303,201903
1.7728863,44.27474,44.35421,1.8247220,201904
1.7212488,46.09546,46.08864,1.7460278,201905
1.6776304,45.06333,44.95500,1.6871348,201906
1.6431248,43.98333,43.75143,1.6361679,201907
1.5814483,54.84286,54.65048,1.5927530,201908
1.4947527,61.05952,58.79000,1.5549163,201909


In [18]:
drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}


In [19]:
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


In [20]:
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}


In [21]:
drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}


In [22]:
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}


In [23]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [24]:
drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}


In [25]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"                      "mrentabilidad_annual"              
 [3] "mcomisiones"                        "mactivos_margen"                   
 [5] "mpasivos_margen"                    "mcuenta_corriente"                 
 [7] "mcaja_ahorro"                       "mcuentas_saldo"                    
 [9] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
[11] "mprestamos_personales"              "mpayroll"                          
[13] "mttarjeta_visa_debitos_automaticos" "mcomisiones_mantenimiento"         
[15] "mtransferencias_recibidas"          "Master_mfinanciacion_limite"       
[17] "Master_msaldototal"                 "Master_mlimitecompra"              
[19] "Master_mconsumototal"               "Master_mpagominimo"                
[21] "Visa_mfinanciacion_limite"          "Visa_msaldototal"                  
[23] "Visa_mlimitecompra"                 "Visa_mconsumototal"                
[25] "Visa_mpagominimo"

In [26]:
# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$DR$metodo <- "rank_simple"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


inicio drift_rank_simple()
mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  mcuenta_corriente  mcaja_ahorro  mcuentas_saldo  mtarjeta_visa_consumo  mtarjeta_master_consumo  mprestamos_personales  mpayroll  mttarjeta_visa_debitos_automaticos  mcomisiones_mantenimiento  mtransferencias_recibidas  Master_mfinanciacion_limite  Master_msaldototal  Master_mlimitecompra  Master_mconsumototal  Master_mpagominimo  Visa_mfinanciacion_limite  Visa_msaldototal  Visa_mlimitecompra  Visa_mconsumototal  Visa_mpagominimo  fin drift_rank_simple()


In [27]:
colnames(dataset)

[1] "numero_de_cliente"                      
 [2] "foto_mes"                               
 [3] "internet"                               
 [4] "cliente_edad"                           
 [5] "cliente_antiguedad"                     
 [6] "cproductos"                             
 [7] "cdescubierto_preacordado"               
 [8] "ctarjeta_visa"                          
 [9] "ctarjeta_visa_transacciones"            
[10] "ctarjeta_master"                        
[11] "ctarjeta_master_transacciones"          
[12] "cprestamos_personales"                  
[13] "cpayroll_trx"                           
[14] "ccomisiones_mantenimiento"              
[15] "ccomisiones_otras"                      
[16] "ccallcenter_transacciones"              
[17] "thomebanking"                           
[18] "chomebanking_transacciones"             
[19] "ctrx_quarter"                           
[20] "Master_status"                          
[21] "Master_Fvencimiento"                    
[22] "Master_fultimo_cierre"                  
[23] "Master_fechaalta"                       
[24] "Master_cconsumos"                       
[25] "Visa_status"                            
[26] "Visa_Fvencimiento"                      
[27] "Visa_fultimo_cierre"                    
[28] "Visa_fechaalta"                         
[29] "Visa_cconsumos"                         
[30] "clase_ternaria"                         
[31] "mrentabilidad_rank"                     
[32] "mrentabilidad_annual_rank"              
[33] "mcomisiones_rank"                       
[34] "mactivos_margen_rank"                   
[35] "mpasivos_margen_rank"                   
[36] "mcuenta_corriente_rank"                 
[37] "mcaja_ahorro_rank"                      
[38] "mcuentas_saldo_rank"                    
[39] "mtarjeta_visa_consumo_rank"             
[40] "mtarjeta_master_consumo_rank"           
[41] "mprestamos_personales_rank"             
[42] "mpayroll_rank"                          
[43] "mttarjeta_visa_debitos_automaticos_rank"
[44] "mcomisiones_mantenimiento_rank"         
[45] "mtransferencias_recibidas_rank"         
[46] "Master_mfinanciacion_limite_rank"       
[47] "Master_msaldototal_rank"                
[48] "Master_mlimitecompra_rank"              
[49] "Master_mconsumototal_rank"              
[50] "Master_mpagominimo_rank"                
[51] "Visa_mfinanciacion_limite_rank"         
[52] "Visa_msaldototal_rank"                  
[53] "Visa_mlimitecompra_rank"                
[54] "Visa_mconsumototal_rank"                
[55] "Visa_mpagominimo_rank"

In [28]:
# se intenta corregir el data drifting utilizando algunos indices financieros

#### 9.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [29]:
if( !require("lubridate")) install.packages("lubridate", repos = "http://cran.us.r-project.org")
require("lubridate")

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [30]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

In [31]:
# Esta es la parte que los alumnos deben desplegar todo su ingenio
# Agregar aqui sus PROPIAS VARIABLES manuales

AgregarVariables_IntraMes <- function(dataset) {
  cat( "inicio AgregarVariables_IntraMes()\n")
  gc(verbose= FALSE)
  # INICIO de la seccion donde se deben hacer cambios con variables nuevas

  # el mes 1,2, ..12
  if( atributos_presentes( c("foto_mes") ))
    dataset[, kmes := foto_mes %% 100]

  # creo un ctr_quarter que tenga en cuenta cuando
  # los clientes hace 3 menos meses que estan
  # ya que seria injusto considerar las transacciones medidas en menor tiempo
  if( atributos_presentes( c("ctrx_quarter") ))
    dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[
      cliente_antiguedad == 3,
      ctrx_quarter_normalizado := ctrx_quarter * 1.2
    ]

   if(atributos_presentes(c("foto_mes")))
    dataset[,foto_mes_formato_fecha := as.Date(paste(substr(dataset$foto_mes,1,4),substr(dataset$foto_mes,5,6),"01",sep='-'))]

  #dataset$foto_mes_formato_fecha <<- as.Date(paste(substr(dataset$foto_mes,1,4),substr(dataset$foto_mes,5,6),"01",sep='-'))

  if(atributos_presentes(c("cantidad_total_transacciones"))){
   auxiliarmenos1 <- dataset[,list(numero_de_cliente,foto_mes_formato_fecha, cantidad_total_transacciones)]
   auxiliarmenos2 <- dataset[,list(numero_de_cliente,foto_mes_formato_fecha,cantidad_total_transacciones)]
   # auxiliarmenos1$foto_mes_formato_fecha <- as.Date(auxiliarmenos1$foto_mes_formato_fecha)
   # auxiliarmenos2$foto_mes_formato_fecha <- as.Date(auxiliarmenos2$foto_mes_formato_fecha)
   auxiliarmenos1$foto_mes_formato_fecha <- auxiliarmenos1$foto_mes_formato_fecha  %m-%  months(1)
   auxiliarmenos2$foto_mes_formato_fecha <- auxiliarmenos2$foto_mes_formato_fecha %m-% months(2)
   auxiliarmenos1$codigo <- paste(auxiliarmenos1$numero_de_cliente,auxiliarmenos1$foto_mes_formato_fecha,sep='-')
   auxiliarmenos2$codigo <- paste(auxiliarmenos2$numero_de_cliente,auxiliarmenos2$foto_mes_formato_fecha,sep='-')

   dataset[, codigo := paste(numero_de_cliente, foto_mes_formato_fecha, sep='-') ]

   dataset[ auxiliarmenos1,
            on = "codigo",
            transaccionesmenos1 := i.cantidad_total_transacciones ]

   dataset[ auxiliarmenos2,
            on = "codigo",
            transaccionesmenos2 := i.cantidad_total_transacciones ]

   dataset[, cantidad_total_transacciones_quarter := rowSums(cbind(cantidad_total_transacciones +
    transaccionesmenos1 + transaccionesmenos2),na.rm=T) ]

   dataset[, codigo := NULL ]
   dataset[, transaccionesmenos1 := NULL ]
   dataset[, transaccionesmenos2 := NULL ]
   dataset[, foto_mes_formato_fecha := NULL ]
   rm(auxiliarmenos1)
   rm(auxiliarmenos2)
  }

  if( atributos_presentes( c("cantidad_total_transacciones_quarter") ))
    dataset[, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 5]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 2]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 3, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 1.2]

  # variable extraida de una tesis de maestria de Irlanda
  if( atributos_presentes( c("mpayroll", "cliente_edad") ))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # se crean los nuevos campos para MasterCard  y Visa,
  #  teniendo en cuenta los NA's
  # varias formas de combinar Visa_status y Master_status
  if( atributos_presentes( c("Master_status", "Visa_status") ))
  {
    dataset[, vm_status01 := pmax(Master_status, Visa_status, na.rm = TRUE)]
    dataset[, vm_status02 := Master_status + Visa_status]

    dataset[, vm_status03 := pmax(
      ifelse(is.na(Master_status), 10, Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status)
    )]

    dataset[, vm_status04 := ifelse(is.na(Master_status), 10, Master_status)
      + ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status05 := ifelse(is.na(Master_status), 10, Master_status)
      + 100 * ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status06 := ifelse(is.na(Visa_status),
      ifelse(is.na(Master_status), 10, Master_status),
      Visa_status
    )]

    dataset[, mv_status07 := ifelse(is.na(Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status),
      Master_status
    )]
  }


  # combino MasterCard y Visa
  if( atributos_presentes( c("Master_mfinanciacion_limite", "Visa_mfinanciacion_limite") ))
    dataset[, vm_mfinanciacion_limite := rowSums(cbind(Master_mfinanciacion_limite, Visa_mfinanciacion_limite), na.rm = TRUE)]

  if( atributos_presentes( c("Master_Fvencimiento", "Visa_Fvencimiento") ))
    dataset[, vm_Fvencimiento := pmin(Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE)]

  if( atributos_presentes( c("Master_Finiciomora", "Visa_Finiciomora") ))
    dataset[, vm_Finiciomora := pmin(Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldototal", "Visa_msaldototal") ))
    dataset[, vm_msaldototal := rowSums(cbind(Master_msaldototal, Visa_msaldototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldopesos", "Visa_msaldopesos") ))
    dataset[, vm_msaldopesos := rowSums(cbind(Master_msaldopesos, Visa_msaldopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldodolares", "Visa_msaldodolares") ))
    dataset[, vm_msaldodolares := rowSums(cbind(Master_msaldodolares, Visa_msaldodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumospesos", "Visa_mconsumospesos") ))
    dataset[, vm_mconsumospesos := rowSums(cbind(Master_mconsumospesos, Visa_mconsumospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumosdolares", "Visa_mconsumosdolares") ))
    dataset[, vm_mconsumosdolares := rowSums(cbind(Master_mconsumosdolares, Visa_mconsumosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mlimitecompra", "Visa_mlimitecompra") ))
    dataset[, vm_mlimitecompra := rowSums(cbind(Master_mlimitecompra, Visa_mlimitecompra), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantopesos", "Visa_madelantopesos") ))
    dataset[, vm_madelantopesos := rowSums(cbind(Master_madelantopesos, Visa_madelantopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantodolares", "Visa_madelantodolares") ))
    dataset[, vm_madelantodolares := rowSums(cbind(Master_madelantodolares, Visa_madelantodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fultimo_cierre", "Visa_fultimo_cierre") ))
    dataset[, vm_fultimo_cierre := pmax(Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagado", "Visa_mpagado") ))
    dataset[, vm_mpagado := rowSums(cbind(Master_mpagado, Visa_mpagado), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagospesos", "Visa_mpagospesos") ))
    dataset[, vm_mpagospesos := rowSums(cbind(Master_mpagospesos, Visa_mpagospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagosdolares", "Visa_mpagosdolares") ))
    dataset[, vm_mpagosdolares := rowSums(cbind(Master_mpagosdolares, Visa_mpagosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fechaalta", "Visa_fechaalta") ))
    dataset[, vm_fechaalta := pmax(Master_fechaalta, Visa_fechaalta, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumototal", "Visa_mconsumototal") ))
    dataset[, vm_mconsumototal := rowSums(cbind(Master_mconsumototal, Visa_mconsumototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cconsumos", "Visa_cconsumos") ))
    dataset[, vm_cconsumos := rowSums(cbind(Master_cconsumos, Visa_cconsumos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cadelantosefectivo", "Visa_cadelantosefectivo") ))
    dataset[, vm_cadelantosefectivo := rowSums(cbind(Master_cadelantosefectivo, Visa_cadelantosefectivo), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagominimo", "Visa_mpagominimo") ))
    dataset[, vm_mpagominimo := rowSums(cbind(Master_mpagominimo, Visa_mpagominimo), na.rm = TRUE)]

  # a partir de aqui juego con la suma de Mastercard y Visa
  if( atributos_presentes( c("Master_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Master_mlimitecompra := Master_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("Visa_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Visa_mlimitecompra := Visa_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldototal", "vm_mlimitecompra") ))
    dataset[, vmr_msaldototal := vm_msaldototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_mlimitecompra") ))
    dataset[, vmr_msaldopesos := vm_msaldopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_msaldototal") ))
    dataset[, vmr_msaldopesos2 := vm_msaldopesos / vm_msaldototal]

  if( atributos_presentes( c("vm_msaldodolares", "vm_mlimitecompra") ))
    dataset[, vmr_msaldodolares := vm_msaldodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldodolares", "vm_msaldototal") ))
    dataset[, vmr_msaldodolares2 := vm_msaldodolares / vm_msaldototal]

  if( atributos_presentes( c("vm_mconsumospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumospesos := vm_mconsumospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumosdolares := vm_mconsumosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantopesos", "vm_mlimitecompra") ))
    dataset[, vmr_madelantopesos := vm_madelantopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantodolares", "vm_mlimitecompra") ))
    dataset[, vmr_madelantodolares := vm_madelantodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagado", "vm_mlimitecompra") ))
    dataset[, vmr_mpagado := vm_mpagado / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mpagospesos := vm_mpagospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mpagosdolares := vm_mpagosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumototal", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumototal := vm_mconsumototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagominimo", "vm_mlimitecompra") ))
    dataset[, vmr_mpagominimo := vm_mpagominimo / vm_mlimitecompra]

  # Aqui debe usted agregar sus propias nuevas variables

  # valvula de seguridad para evitar valores infinitos
  # paso los infinitos a NULOS
  infinitos <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.infinite(get(.name)))]
  )

  infinitos_qty <- sum(unlist(infinitos))
  if (infinitos_qty > 0) {
    cat(
      "ATENCION, hay", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n"
    )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  # valvula de seguridad para evitar valores NaN  que es 0/0
  # paso los NaN a 0 , decision polemica si las hay
  # se invita a asignar un valor razonable segun la semantica del campo creado
  nans <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.nan(get(.name)))]
  )

  nans_qty <- sum(unlist(nans))
  if (nans_qty > 0) {
    cat(
      "ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    )

    cat("Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

  cat( "fin AgregarVariables_IntraMes()\n")
}


In [32]:
# agrego las variables intra mes
notificar("Agegar Variables")
AgregarVariables_IntraMes(dataset)
notificar("Fin agregar Variables")

inicio AgregarVariables_IntraMes()
fin AgregarVariables_IntraMes()


In [33]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"                      
 [2] "foto_mes"                               
 [3] "internet"                               
 [4] "cliente_edad"                           
 [5] "cliente_antiguedad"                     
 [6] "cproductos"                             
 [7] "cdescubierto_preacordado"               
 [8] "ctarjeta_visa"                          
 [9] "ctarjeta_visa_transacciones"            
[10] "ctarjeta_master"                        
[11] "ctarjeta_master_transacciones"          
[12] "cprestamos_personales"                  
[13] "cpayroll_trx"                           
[14] "ccomisiones_mantenimiento"              
[15] "ccomisiones_otras"                      
[16] "ccallcenter_transacciones"              
[17] "thomebanking"                           
[18] "chomebanking_transacciones"             
[19] "ctrx_quarter"                           
[20] "Master_status"                          
[21] "Master_Fvencimiento"                    
[22] "Master_fultimo_cierre"                  
[23] "Master_fechaalta"                       
[24] "Master_cconsumos"                       
[25] "Visa_status"                            
[26] "Visa_Fvencimiento"                      
[27] "Visa_fultimo_cierre"                    
[28] "Visa_fechaalta"                         
[29] "Visa_cconsumos"                         
[30] "clase_ternaria"                         
[31] "mrentabilidad_rank"                     
[32] "mrentabilidad_annual_rank"              
[33] "mcomisiones_rank"                       
[34] "mactivos_margen_rank"                   
[35] "mpasivos_margen_rank"                   
[36] "mcuenta_corriente_rank"                 
[37] "mcaja_ahorro_rank"                      
[38] "mcuentas_saldo_rank"                    
[39] "mtarjeta_visa_consumo_rank"             
[40] "mtarjeta_master_consumo_rank"           
[41] "mprestamos_personales_rank"             
[42] "mpayroll_rank"                          
[43] "mttarjeta_visa_debitos_automaticos_rank"
[44] "mcomisiones_mantenimiento_rank"         
[45] "mtransferencias_recibidas_rank"         
[46] "Master_mfinanciacion_limite_rank"       
[47] "Master_msaldototal_rank"                
[48] "Master_mlimitecompra_rank"              
[49] "Master_mconsumototal_rank"              
[50] "Master_mpagominimo_rank"                
[51] "Visa_mfinanciacion_limite_rank"         
[52] "Visa_msaldototal_rank"                  
[53] "Visa_mlimitecompra_rank"                
[54] "Visa_mconsumototal_rank"                
[55] "Visa_mpagominimo_rank"                  
[56] "kmes"                                   
[57] "ctrx_quarter_normalizado"               
[58] "foto_mes_formato_fecha"                 
[59] "vm_status01"                            
[60] "vm_status02"                            
[61] "vm_status03"                            
[62] "vm_status04"                            
[63] "vm_status05"                            
[64] "vm_status06"                            
[65] "mv_status07"                            
[66] "vm_Fvencimiento"                        
[67] "vm_fultimo_cierre"                      
[68] "vm_fechaalta"                           
[69] "vm_cconsumos"

#### 9.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

In [34]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm



In [35]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$FE_rf$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


In [36]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 25
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [37]:
notificar("🚀 Comienza entrenamiento RF")

In [38]:
# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  202003 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202004 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202005 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202006 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202007 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202008 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202009 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202010 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

In [39]:
notificar("🚀 fin entrenamiento RF")

In [40]:
ncol(dataset)
colnames(dataset)

[1] 570

[1] "numero_de_cliente"                      
  [2] "foto_mes"                               
  [3] "internet"                               
  [4] "cliente_edad"                           
  [5] "cliente_antiguedad"                     
  [6] "cproductos"                             
  [7] "cdescubierto_preacordado"               
  [8] "ctarjeta_visa"                          
  [9] "ctarjeta_visa_transacciones"            
 [10] "ctarjeta_master"                        
 [11] "ctarjeta_master_transacciones"          
 [12] "cprestamos_personales"                  
 [13] "cpayroll_trx"                           
 [14] "ccomisiones_mantenimiento"              
 [15] "ccomisiones_otras"                      
 [16] "ccallcenter_transacciones"              
 [17] "thomebanking"                           
 [18] "chomebanking_transacciones"             
 [19] "ctrx_quarter"                           
 [20] "Master_status"                          
 [21] "Master_Fvencimiento"                    
 [22] "Master_fultimo_cierre"                  
 [23] "Master_fechaalta"                       
 [24] "Master_cconsumos"                       
 [25] "Visa_status"                            
 [26] "Visa_Fvencimiento"                      
 [27] "Visa_fultimo_cierre"                    
 [28] "Visa_fechaalta"                         
 [29] "Visa_cconsumos"                         
 [30] "clase_ternaria"                         
 [31] "mrentabilidad_rank"                     
 [32] "mrentabilidad_annual_rank"              
 [33] "mcomisiones_rank"                       
 [34] "mactivos_margen_rank"                   
 [35] "mpasivos_margen_rank"                   
 [36] "mcuenta_corriente_rank"                 
 [37] "mcaja_ahorro_rank"                      
 [38] "mcuentas_saldo_rank"                    
 [39] "mtarjeta_visa_consumo_rank"             
 [40] "mtarjeta_master_consumo_rank"           
 [41] "mprestamos_personales_rank"             
 [42] "mpayroll_rank"                          
 [43] "mttarjeta_visa_debitos_automaticos_rank"
 [44] "mcomisiones_mantenimiento_rank"         
 [45] "mtransferencias_recibidas_rank"         
 [46] "Master_mfinanciacion_limite_rank"       
 [47] "Master_msaldototal_rank"                
 [48] "Master_mlimitecompra_rank"              
 [49] "Master_mconsumototal_rank"              
 [50] "Master_mpagominimo_rank"                
 [51] "Visa_mfinanciacion_limite_rank"         
 [52] "Visa_msaldototal_rank"                  
 [53] "Visa_mlimitecompra_rank"                
 [54] "Visa_mconsumototal_rank"                
 [55] "Visa_mpagominimo_rank"                  
 [56] "kmes"                                   
 [57] "ctrx_quarter_normalizado"               
 [58] "foto_mes_formato_fecha"                 
 [59] "vm_status01"                            
 [60] "vm_status02"                            
 [61] "vm_status03"                            
 [62] "vm_status04"                            
 [63] "vm_status05"                            
 [64] "vm_status06"                            
 [65] "mv_status07"                            
 [66] "vm_Fvencimiento"                        
 [67] "vm_fultimo_cierre"                      
 [68] "vm_fechaalta"                           
 [69] "vm_cconsumos"                           
 [70] "entrenamiento"                          
 [71] "rf_001_015"                             
 [72] "rf_001_007"                             
 [73] "rf_001_022"                             
 [74] "rf_001_008"                             
 [75] "rf_001_006"                             
 [76] "rf_001_016"                             
 [77] "rf_001_002"                             
 [78] "rf_001_003"                             
 [79] "rf_001_011"                             
 [80] "rf_001_024"                             
 [81] "rf_001_010"                             
 [82] "rf_001_001"                             
 [83] "rf_001_017"                             
 [84] "rf_001_021"

In [41]:
# No se implementa Feature Engineering a partir de Random Forest

#### 9.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [42]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [43]:
ncol(dataset)
colnames(dataset)

[1] 2838

[1] "numero_de_cliente"                             
   [2] "foto_mes"                                      
   [3] "internet"                                      
   [4] "cliente_edad"                                  
   [5] "cliente_antiguedad"                            
   [6] "cproductos"                                    
   [7] "cdescubierto_preacordado"                      
   [8] "ctarjeta_visa"                                 
   [9] "ctarjeta_visa_transacciones"                   
  [10] "ctarjeta_master"                               
  [11] "ctarjeta_master_transacciones"                 
  [12] "cprestamos_personales"                         
  [13] "cpayroll_trx"                                  
  [14] "ccomisiones_mantenimiento"                     
  [15] "ccomisiones_otras"                             
  [16] "ccallcenter_transacciones"                     
  [17] "thomebanking"                                  
  [18] "chomebanking_transacciones"                    
  [19] "ctrx_quarter"                                  
  [20] "Master_status"                                 
  [21] "Master_Fvencimiento"                           
  [22] "Master_fultimo_cierre"                         
  [23] "Master_fechaalta"                              
  [24] "Master_cconsumos"                              
  [25] "Visa_status"                                   
  [26] "Visa_Fvencimiento"                             
  [27] "Visa_fultimo_cierre"                           
  [28] "Visa_fechaalta"                                
  [29] "Visa_cconsumos"                                
  [30] "clase_ternaria"                                
  [31] "mrentabilidad_rank"                            
  [32] "mrentabilidad_annual_rank"                     
  [33] "mcomisiones_rank"                              
  [34] "mactivos_margen_rank"                          
  [35] "mpasivos_margen_rank"                          
  [36] "mcuenta_corriente_rank"                        
  [37] "mcaja_ahorro_rank"                             
  [38] "mcuentas_saldo_rank"                           
  [39] "mtarjeta_visa_consumo_rank"                    
  [40] "mtarjeta_master_consumo_rank"                  
  [41] "mprestamos_personales_rank"                    
  [42] "mpayroll_rank"                                 
  [43] "mttarjeta_visa_debitos_automaticos_rank"       
  [44] "mcomisiones_mantenimiento_rank"                
  [45] "mtransferencias_recibidas_rank"                
  [46] "Master_mfinanciacion_limite_rank"              
  [47] "Master_msaldototal_rank"                       
  [48] "Master_mlimitecompra_rank"                     
  [49] "Master_mconsumototal_rank"                     
  [50] "Master_mpagominimo_rank"                       
  [51] "Visa_mfinanciacion_limite_rank"                
  [52] "Visa_msaldototal_rank"                         
  [53] "Visa_mlimitecompra_rank"                       
  [54] "Visa_mconsumototal_rank"                       
  [55] "Visa_mpagominimo_rank"                         
  [56] "kmes"                                          
  [57] "ctrx_quarter_normalizado"                      
  [58] "foto_mes_formato_fecha"                        
  [59] "vm_status01"                                   
  [60] "vm_status02"                                   
  [61] "vm_status03"                                   
  [62] "vm_status04"                                   
  [63] "vm_status05"                                   
  [64] "vm_status06"                                   
  [65] "mv_status07"                                   
  [66] "vm_Fvencimiento"                               
  [67] "vm_fultimo_cierre"                             
  [68] "vm_fechaalta"                                  
  [69] "vm_cconsumos"                                  
  [70] "entrenamiento"                                 
  [71] "rf_001_015"                                    
  [72] "rf_001_007"        

#### 9.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [44]:
# No se implementa la reduccion de la dimensionalidad con canaritos

In [45]:
VPOS_CORTE <- c()

fganancia_lgbm_meseta <- function(probs, datos) {
  vlabels <- get_field(datos, "label")
  vpesos <- get_field(datos, "weight")

  tbl <- as.data.table(list(
    "prob" = probs,
    "gan" = ifelse(vlabels == 1 & vpesos > 1, PARAM$CN$train$gan1, PARAM$CN$train$gan0)
  ))

  setorder(tbl, -prob)
  tbl[, posicion := .I]
  tbl[, gan_acum := cumsum(gan)]
  setorder(tbl, -gan_acum) # voy por la meseta

  gan <- mean(tbl[1:500, gan_acum]) # meseta de tamaño 500

  pos_meseta <- tbl[1:500, median(posicion)]
  VPOS_CORTE <<- c(VPOS_CORTE, pos_meseta)

  return(list(
    "name" = "ganancia",
    "value" = gan,
    "higher_better" = TRUE
  ))
}


In [46]:
# Elimina del dataset las variables que estan por debajo
#  de la capa geologica de canaritos
# se llama varias veces, luego de agregar muchas variables nuevas,
#  para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1

campitos <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

CanaritosAsesinos <- function(
  canaritos_ratio,
  canaritos_desvios,
  canaritos_semilla) {

  cat( "inicio CanaritosAsesinos()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$CN$train$clase01_valor1,
      clase01 := 1L ]

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  for (i in 1:(ncol(dataset) * canaritos_ratio)) {
    dataset[, paste0("canarito", i) := runif(nrow(dataset))]
  }

  campos_buenos <- setdiff(
    colnames(dataset),
    c( campitos, "clase01")
  )

  azar <- runif(nrow(dataset))

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$CN$train$training &
      (clase01 == 1 | azar < PARAM$CN$train$undersampling))]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    weight = dataset[
      entrenamiento == TRUE,
      ifelse(clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )

  dvalid <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$CN$train$validation, campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$CN$train$validation, clase01],
    weight = dataset[
      foto_mes %in% PARAM$CN$train$validation,
      ifelse( clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )


  param <- list(
    objective = "binary",
    metric = "custom",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    seed = canaritos_semilla,
    max_depth = -1, # -1 significa no limitar,  por ahora lo dejo fijo
    min_gain_to_split = 0.0, # por ahora, lo dejo fijo
    lambda_l1 = 0.0, # por ahora, lo dejo fijo
    lambda_l2 = 0.0, # por ahora, lo dejo fijo
    max_bin = 31, # por ahora, lo dejo fijo
    num_iterations = 9999, # un numero grande, lo limita early_stopping_rounds
    force_row_wise = TRUE, # para que los alumnos no se atemoricen con  warning
    learning_rate = 0.065,
    feature_fraction = 1.0, # lo seteo en 1
    min_data_in_leaf = 260,
    num_leaves = 60,
    early_stopping_rounds = 200,
    num_threads = 1
  )

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  modelo <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalid),
    eval = fganancia_lgbm_meseta,
    param = param,
    verbose = -100
  )

  tb_importancia <- lgb.importance(model = modelo)
  tb_importancia[, pos := .I]

  fwrite(tb_importancia,
    file = paste0("impo_", GVEZ, ".txt"),
    sep = "\t"
  )

  GVEZ <<- GVEZ + 1

  umbral <- tb_importancia[
    Feature %like% "canarito",
    median(pos) + canaritos_desvios * sd(pos)
  ] # Atencion corto en la mediana mas desvios!!

  col_utiles <- tb_importancia[
    pos < umbral & !(Feature %like% "canarito"),
    Feature
  ]

  col_utiles <- unique(c(
    col_utiles,
    c(campitos, "mes")
  ))

  col_inutiles <- setdiff(colnames(dataset), col_utiles)

  dataset[, (col_inutiles) := NULL]

  cat( "fin CanaritosAsesinos()\n")

  return( tb_importancia )
}


In [47]:
# Estos DOS parametros son los que se deben modificar
PARAM$CN$ratio <- 0.2
PARAM$CN$desvios <- 2


# Parametros quasi fijos
# Parametros de un LightGBM que se genera para estimar la column importance
PARAM$CN$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$CN$train$positivos <- c( "BAJA+2")
PARAM$CN$train$training <- c( 202101, 202102, 202103)
PARAM$CN$train$validation <- c( 202105 )
PARAM$CN$train$undersampling <- 0.1
PARAM$CN$train$gan1 <- 117000
PARAM$CN$train$gan0 <-  -3000

In [48]:
# la llamada a Canaritos Asesinos
notificar("inicio canaritosAsesinos")
tb_importancia <- CanaritosAsesinos(
  canaritos_ratio = PARAM$CN$ratio,
  canaritos_desvios = PARAM$CN$desvios,
  canaritos_semilla = PARAM$semilla_primigenia
)
notificar("Fin canaritosAsesinos")

inicio CanaritosAsesinos()
fin CanaritosAsesinos()


In [49]:
# grabo la importancia, ver el archivo directamente en la carpeta

fwrite( tb_importancia,
  file="canaritos.txt",
  sep="\t"
)

In [50]:
# verifico
ncol(dataset)
colnames(dataset)

[1] 324

[1] "numero_de_cliente"                             
  [2] "foto_mes"                                      
  [3] "cliente_edad"                                  
  [4] "cliente_antiguedad"                            
  [5] "cproductos"                                    
  [6] "cdescubierto_preacordado"                      
  [7] "ctarjeta_visa_transacciones"                   
  [8] "cprestamos_personales"                         
  [9] "cpayroll_trx"                                  
 [10] "ccomisiones_mantenimiento"                     
 [11] "ccomisiones_otras"                             
 [12] "ccallcenter_transacciones"                     
 [13] "chomebanking_transacciones"                    
 [14] "ctrx_quarter"                                  
 [15] "Master_Fvencimiento"                           
 [16] "Master_fultimo_cierre"                         
 [17] "Master_fechaalta"                              
 [18] "Visa_status"                                   
 [19] "Visa_Fvencimiento"                             
 [20] "Visa_fultimo_cierre"                           
 [21] "Visa_fechaalta"                                
 [22] "Visa_cconsumos"                                
 [23] "clase_ternaria"                                
 [24] "mrentabilidad_rank"                            
 [25] "mrentabilidad_annual_rank"                     
 [26] "mcomisiones_rank"                              
 [27] "mactivos_margen_rank"                          
 [28] "mpasivos_margen_rank"                          
 [29] "mcuenta_corriente_rank"                        
 [30] "mcaja_ahorro_rank"                             
 [31] "mcuentas_saldo_rank"                           
 [32] "mtarjeta_visa_consumo_rank"                    
 [33] "mtarjeta_master_consumo_rank"                  
 [34] "mprestamos_personales_rank"                    
 [35] "mpayroll_rank"                                 
 [36] "mcomisiones_mantenimiento_rank"                
 [37] "mtransferencias_recibidas_rank"                
 [38] "Master_mfinanciacion_limite_rank"              
 [39] "Master_msaldototal_rank"                       
 [40] "Master_mlimitecompra_rank"                     
 [41] "Master_mconsumototal_rank"                     
 [42] "Master_mpagominimo_rank"                       
 [43] "Visa_msaldototal_rank"                         
 [44] "Visa_mlimitecompra_rank"                       
 [45] "Visa_mconsumototal_rank"                       
 [46] "Visa_mpagominimo_rank"                         
 [47] "kmes"                                          
 [48] "ctrx_quarter_normalizado"                      
 [49] "vm_status01"                                   
 [50] "vm_status03"                                   
 [51] "vm_Fvencimiento"                               
 [52] "vm_fechaalta"                                  
 [53] "vm_cconsumos"                                  
 [54] "rf_001_015"                                    
 [55] "rf_001_008"                                    
 [56] "rf_001_005"                                    
 [57] "rf_001_012"                                    
 [58] "rf_002_024"                                    
 [59] "rf_002_007"                                    
 [60] "rf_002_014"                                    
 [61] "rf_003_007"                                    
 [62] "rf_004_006"                                    
 [63] "rf_005_001"                                    
 [64] "rf_006_007"                                    
 [65] "rf_007_010"                                    
 [66] "rf_007_009"                                    
 [67] "rf_007_020"                                    
 [68] "rf_007_007"                                    
 [69] "rf_009_012"                                    
 [70] "rf_011_012"                                    
 [71] "rf_012_024"                                    
 [72] "rf_013_021"                                    
 [73] "rf_013_009"                        

### 9.3.2 Modelado

#### 9.3.2.1 Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 201901, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 201901, 202105 ]  donde se consideran el 100% de los CONTINUA

In [51]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105
)


PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [52]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [53]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [54]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [55]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 33053

####  9.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [56]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [86]:
# un Analista Jr  debe poder animarse a hacer 100 iteraciones
PARAM$hipeparametertuning$num_interations <- 30

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,

  num_iterations= 9999,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 400
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower= 2L, upper= 256L),
  makeIntegerParam("min_data_in_leaf", lower= 2L, upper= 8192L),
  makeNumericParam("feature_fraction", lower= 0.1, upper= 0.9),
  makeNumericParam("learning_rate", lower= 0.01, upper= 0.5)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [87]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [88]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [89]:
notificar("🚀 Comienza entrenamiento LGBM")

In [90]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Sun Aug 24 19:47:46 2025 AUC 0.929149473956066

[mbo] 25: num_leaves=256; min_data_in_leaf=3857; feature_fraction=0.209; learning_rate=0.01 : y = 0.929 : 215.9 secs : infill_ei

Saved the current state after iteration 26 in the file HT.RDATA.

Sun Aug 24 19:51:44 2025 AUC 0.928061505649025

[mbo] 26: num_leaves=255; min_data_in_leaf=7366; feature_fraction=0.9; learning_rate=0.01 : y = 0.928 : 234.9 secs : infill_ei

Sun Aug 24 19:55:22 2025 AUC 0.929445678182001

[mbo] 27: num_leaves=157; min_data_in_leaf=2756; feature_fraction=0.344; learning_rate=0.01 : y = 0.929 : 217.2 secs : infill_ei

Sun Aug 24 20:00:00 2025 AUC 0.927738676993372

[mbo] 28: num_leaves=256; min_data_in_leaf=4071; feature_fraction=0.9; learning_rate=0.0101 : y = 0.928 : 277.5 secs : infill_ei

Saved the current state after iteration 29 in the file HT.RDATA.

Sun Aug 24 20:04:26 2025 AUC 0.928305711827656

[mbo] 29: num_leaves=256; min_data_in_leaf=6; feature_fraction=0.107; learning_rate=0.0102 : y = 0.928 : 263.7

In [91]:
notificar("🚀 Fin entrenamiento LGBM")

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [93]:
library(mlrMBO)
# 1) si ya existe en la sesión, úsalo; si no, intenta continuar desde el estado guardado
bayesiana_salida <- get0("bayesiana_salida", inherits = TRUE)

if (is.null(bayesiana_salida)) {
  if (file.exists("HT.RDATA")) {
    # OJO: esto retorna el objeto resultado; además actualiza el archivo de estado
    bayesiana_salida <- mboContinue("HT.RDATA")
  } else {
    stop("No existe 'bayesiana_salida' en memoria y no encontré HT.RDATA para continuar.")
  }
}


In [101]:
suppressPackageStartupMessages({
  library(data.table)
})

# --- 1) Reconstrucción robusta del opt.path -> data.table ---
tb_bayesiana <- NULL
if (exists("bayesiana_salida") && !is.null(bayesiana_salida$opt.path)) {
  tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
} else if (file.exists("HT.RDATA")) {
  objs <- load("HT.RDATA")
  # intenta varias rutas posibles
  if (exists("bayesiana_salida") && !is.null(bayesiana_salida$opt.path)) {
    tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
  } else if (exists("opt.state") && !is.null(opt.state$opt.path)) {
    tb_bayesiana <- as.data.table(opt.state$opt.path)
  } else {
    # fallback: primer objeto que tenga $opt.path
    for (nm in objs) {
      obj <- get(nm, inherits = FALSE)
      if (is.list(obj) && "opt.path" %in% names(obj)) {
        tb_bayesiana <- as.data.table(obj$opt.path)
        break
      }
    }
  }
}
stopifnot(!is.null(tb_bayesiana))

# --- 2) Identificar la columna objetivo disponible ---
# patrones comunes: y, y1, y_1, auc, AUC, objective, mlr.y
cand_obj <- names(tb_bayesiana)[
  grepl("^y($|[0-9_])", names(tb_bayesiana), ignore.case = TRUE) |
  grepl("^auc$", names(tb_bayesiana), ignore.case = TRUE) |
  grepl("^objective$", names(tb_bayesiana), ignore.case = TRUE) |
  grepl("^mlr\\.y$", names(tb_bayesiana), ignore.case = TRUE)
]

if (length(cand_obj) == 0L) {
  stop("No encuentro columna de métrica objetivo (y/auc/objective) en opt.path.")
}

# si hay varias (p.ej., y, y1), quedate con la primera no vacía
obj_col <- NULL
for (c in cand_obj) {
  if (is.numeric(tb_bayesiana[[c]]) && any(is.finite(tb_bayesiana[[c]]))) {
    obj_col <- c
    break
  }
}
if (is.null(obj_col)) stop("No hay métrica objetivo numérica y finita en opt.path.")

# filtrar filas válidas
tb_bayesiana <- tb_bayesiana[is.finite(get(obj_col))]

# --- 3) Ordenar por mejor métrica y desempatar por dob si existe ---
ord_cols <- c(obj_col, "dob")
ord_cols <- intersect(ord_cols, names(tb_bayesiana))
ord_dir  <- c(-1L, 1L)[seq_along(ord_cols)]
setorderv(tb_bayesiana, ord_cols, ord_dir)

# --- 4) Guardar log (opcional) ---
fwrite(tb_bayesiana, file = "BO_log.txt", sep = "\t")

# --- 5) Extraer mejores hiperparámetros existentes ---
cols_posibles <- c("num_leaves","min_data_in_leaf","feature_fraction","learning_rate",
                   "bagging_fraction","bagging_freq","max_depth","min_gain_to_split",
                   "lambda_l1","lambda_l2","min_sum_hessian_in_leaf",
                   "num_iterations","nrounds","max_bin")
cols_hp <- intersect(cols_posibles, names(tb_bayesiana))

best <- tb_bayesiana[1, ..cols_hp]
if (nrow(best) == 0L) {
  stop("No encuentro columnas de hiperparámetros en opt.path. Revisá los nombres reales de tus params.")
}

# --- 6) Dejar en PARAM para que tu script siga tal cual ---
if (!exists("PARAM")) PARAM <- list()
if (is.null(PARAM$out)) PARAM$out <- list()
if (is.null(PARAM$out$lgbm)) PARAM$out$lgbm <- list()
PARAM$out$lgbm$mejores_hiperparametros <- best

print(PARAM$out$lgbm$mejores_hiperparametros)



   num_leaves min_data_in_leaf feature_fraction learning_rate num_iterations
        <int>            <int>            <num>         <num>          <int>
1:        172             1874        0.2681796    0.01002921            775


In [102]:
if ("nrounds" %in% names(best) && !"num_iterations" %in% names(best)) {
  setnames(best, "nrounds", "num_iterations")
  PARAM$out$lgbm$mejores_hiperparametros <- best
}


In [98]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

ERROR: Error in setorderv(x, cols, order, na.last): some columns are not in the data.table: [y, num_iterations]


In [113]:
suppressPackageStartupMessages({
  library(data.table)
  library(mlrMBO)
  library(ParamHelpers)
})

# === OptPath robusto (funciona aunque hayas interrumpido) ===
get_optpath <- function() {
  if (exists("bayesiana_salida") && !is.null(bayesiana_salida$opt.path)) return(bayesiana_salida$opt.path)
  if (file.exists("HT.RDATA")) {
    objs <- load("HT.RDATA")
    if (exists("bayesiana_salida") && !is.null(bayesiana_salida$opt.path)) return(bayesiana_salida$opt.path)
    if (exists("opt.state") && !is.null(opt.state$opt.path)) return(opt.state$opt.path)
    for (nm in objs) {
      obj <- get(nm, inherits = FALSE)
      if (is.list(obj) && "opt.path" %in% names(obj)) return(obj$opt.path)
    }
  }
  stop("No pude encontrar un opt.path válido.")
}
op <- get_optpath()

# === Construir tabla de hiperparámetros + métrica ===
X <- as.data.table(getOptPathX(op))         # hiperparámetros probados
Y <- as.numeric(getOptPathY(op))            # métrica (AUC)
stopifnot(nrow(X) == length(Y))
tb_bayesiana <- copy(X)
tb_bayesiana[, y := Y]

# normalizar nombre de iteraciones si vino como nrounds
if ("nrounds" %in% names(tb_bayesiana) && !"num_iterations" %in% names(tb_bayesiana)) {
  setnames(tb_bayesiana, "nrounds", "num_iterations")
}
# si no existe num_iterations, crearla (NA) para no romper tu orden y selección
if (!"num_iterations" %in% names(tb_bayesiana)) {
  tb_bayesiana[, num_iterations := NA_integer_]
}

# === Ordenar por mejor AUC y desempatar por num_iterations si existe ===
tb_bayesiana <- tb_bayesiana[is.finite(y)]
# si todas num_iterations son NA, ordena solo por y
if (all(is.na(tb_bayesiana$num_iterations))) {
  setorder(tb_bayesiana, -y)
} else {
  setorder(tb_bayesiana, -y, -num_iterations)
}

# === Guardar log opcional ===
fwrite(tb_bayesiana, file = "BO_log.txt", sep = "\t")

# === Extraer mejores hiperparámetros (usa solo los que existan) ===
cols_hp <- intersect(
  c("num_leaves","min_data_in_leaf","feature_fraction","learning_rate","num_iterations"),
  names(tb_bayesiana)
)
best <- tb_bayesiana[1, ..cols_hp]

# === Dejar en PARAM como espera tu script ===
if (!exists("PARAM")) PARAM <- list()
if (is.null(PARAM$out)) PARAM$out <- list()
if (is.null(PARAM$out$lgbm)) PARAM$out$lgbm <- list()
PARAM$out$lgbm$mejores_hiperparametros <- best

print(PARAM$out$lgbm$mejores_hiperparametros)



   num_leaves min_data_in_leaf feature_fraction learning_rate num_iterations
        <int>            <int>            <num>         <num>          <int>
1:        172             1874        0.2681796    0.01002921             NA


### 9.3.3 Produccion

In [107]:
print(names(as.data.frame(bayesiana_salida$opt.path)))
str(as.data.frame(bayesiana_salida$opt.path)[, 1:20])  # primeras 20 columnas
summary(getOptPathY(bayesiana_salida$opt.path))


character(0)


ERROR: Error in `[.data.frame`(as.data.frame(bayesiana_salida$opt.path), , 1:20): undefined columns selected


#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [109]:
PARAM$trainingstrategy$final_train <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107
)


dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 910211

##### Final Training Hyperparameters

In [110]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

ERROR: Error in setorderv(x, cols, order, na.last): some columns are not in the data.table: [y, num_iterations]


In [114]:
suppressPackageStartupMessages(library(data.table))

# reconstruyo tabla desde opt.path
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# agrego columna y si no existe (desde getOptPathY)
if (!"y" %in% names(tb_bayesiana)) {
  tb_bayesiana[, y := as.numeric(mlrMBO::getOptPathY(bayesiana_salida$opt.path))]
}

# si no existe num_iterations, lo creo como NA
if (!"num_iterations" %in% names(tb_bayesiana)) {
  tb_bayesiana[, num_iterations := NA_integer_]
}

# ordeno por mejor AUC
tb_bayesiana <- tb_bayesiana[is.finite(y)]
setorder(tb_bayesiana, -y)

# guardo log
fwrite(tb_bayesiana, file = "BO_log.txt", sep = "\t")

# selecciono hiperparámetros y HARDCODEO num_iterations = 775
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1,
  .(num_leaves, min_data_in_leaf)
][, num_iterations := 775L]

print(PARAM$out$lgbm$mejores_hiperparametros)


ERROR: Error: 'getOptPathY' is not an exported object from 'namespace:mlrMBO'


In [115]:
# Toma el mejor set ya guardado
best <- PARAM$out$lgbm$mejores_hiperparametros[1]

# Parámetros finales LightGBM
param_final <- list(
  objective        = "binary",
  metric           = "auc",
  boosting         = "gbdt",
  learning_rate    = as.numeric(best$learning_rate),
  num_leaves       = as.integer(best$num_leaves),
  min_data_in_leaf = as.integer(best$min_data_in_leaf),
  feature_fraction = as.numeric(best$feature_fraction),
  bagging_fraction = 1.0,
  bagging_freq     = 0,
  max_bin          = 255,
  seed             = 102191,
  verbosity        = -1
)

# Si tu pipeline usa nrounds aparte:
num_boost_round <- if ("num_iterations" %in% names(best) && !is.na(best$num_iterations)) {
  as.integer(best$num_iterations)
} else {
  775L  # fallback que definiste
}


In [116]:
param_final

$objective
[1] "binary"

$metric
[1] "auc"

$boosting
[1] "gbdt"

$learning_rate
[1] 0.01002921

$num_leaves
[1] 172

$min_data_in_leaf
[1] 1874

$feature_fraction
[1] 0.2681796

$bagging_fraction
[1] 1

$bagging_freq
[1] 0

$max_bin
[1] 255

$seed
[1] 102191

$verbosity
[1] -1

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [117]:
PARAM
semillerio <- 20  # cantidad de semillas

$semilla_primigenia
[1] 102191

$experimento
[1] 1236

$dataset
[1] "analistajr_competencia_2025.csv.gz"

$CA
$CA$metodo
[1] "MachineLearning"


$DR
$DR$metodo
[1] "rank_simple"


$FE_rf
$FE_rf$arbolitos
[1] 20

$FE_rf$hojas_por_arbol
[1] 25

$FE_rf$datos_por_hoja
[1] 100

$FE_rf$mtry_ratio
[1] 0.2

$FE_rf$train
$FE_rf$train$clase01_valor1
[1] "BAJA+2" "BAJA+1"

$FE_rf$train$training
[1] 202101 202102 202103


$FE_rf$lgb_param
$FE_rf$lgb_param$num_iterations
[1] 20

$FE_rf$lgb_param$num_leaves
[1] 25

$FE_rf$lgb_param$min_data_in_leaf
[1] 100

$FE_rf$lgb_param$feature_fraction_bynode
[1] 0.2

$FE_rf$lgb_param$boosting
[1] "rf"

$FE_rf$lgb_param$bagging_fraction
[1] 0.6321206

$FE_rf$lgb_param$bagging_freq
[1] 1

$FE_rf$lgb_param$feature_fraction
[1] 1

$FE_rf$lgb_param$max_bin
[1] 31

$FE_rf$lgb_param$objective
[1] "binary"

$FE_rf$lgb_param$first_metric_only
[1] TRUE

$FE_rf$lgb_param$boost_from_average
[1] TRUE

$FE_rf$lgb_param$feature_pre_filter
[1] FALSE

$FE_rf$lgb_param$force_ro

In [123]:

if(!require("primes")) install.packages("primes")
require("primes")

In [124]:
PARAM$FT$semillerio <- 20  # cantidad de semillas

In [125]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$FT$semillerio)]

cat( PARAM$FT$semillas)

586973 993977 818581 923141 240763 277177 500953 849311 288181 995381 496609 134171 305749 455827 440717 597853 871349 438287 379787 267833

In [126]:


notificar("Semillero finalizado")

ERROR: Error in parse(text = input): <text>:5:1: unexpected symbol
4: for( sem in PARAM
5: semillas
   ^


In [127]:
dir.create("modelos", showWarnings =FALSE)

primero <- TRUE
for( sem in PARAM$FT$semillas)
{
  nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
  if( !file.exists(nombre_arch) )
  {
    param_final$seed <- sem

    set.seed(sem, kind = "L'Ecuyer-CMRG")
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    lgb.save(final_model, nombre_arch) # grabo el modelo"

    # grabo la primer importancia de variables
    #  Natalia : da lo mismo cual se guarda
    if( primero)
    {
      primero <- FALSE
      tb_importancia <- as.data.table(lgb.importance(final_model))
      archivo_importancia <- "impo.txt"

      fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
      )
    }
 }
}

#### Scoring

Aplico el modelo final a los datos del futuro

In [128]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [130]:
# aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2757897,147.3,5934198,317.0,5934198,317.0
Vcells,249502279,1903.6,788011861,6012.1,2835769564,21635.3


In [131]:

# veo que hay en tb_prediccion
tb_prediccion

numero_de_cliente,prob,prob_1,prob_2,prob_3,prob_4,prob_5,prob_6,prob_7,prob_8,⋯,prob_11,prob_12,prob_13,prob_14,prob_15,prob_16,prob_17,prob_18,prob_19,prob_20
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,0.003998780,0.003840677,0.004024733,0.004175728,0.004032548,0.004055868,0.004188465,0.003992408,0.004129303,⋯,0.003925274,0.003975870,0.004083692,0.004064275,0.003955173,0.003832982,0.003931658,0.004141153,0.004042510,0.003807098
29187730,0.003420208,0.003413636,0.003414763,0.003423943,0.003426226,0.003422920,0.003415560,0.003434780,0.003420561,⋯,0.003420812,0.003413979,0.003415655,0.003419219,0.003414785,0.003418292,0.003421099,0.003421002,0.003418389,0.003424083
29187961,0.004140319,0.004260579,0.004219341,0.004028092,0.004108071,0.004038822,0.004098074,0.004102728,0.004065307,⋯,0.004071519,0.004196751,0.004051948,0.004271611,0.004244180,0.004072376,0.004174430,0.004117201,0.004124018,0.004232974
29193101,0.004254421,0.004182308,0.004112348,0.004394621,0.004237943,0.004271658,0.004447915,0.004267257,0.004313949,⋯,0.004020841,0.004249943,0.004465825,0.004382060,0.004401278,0.004183131,0.004119841,0.004060886,0.004239512,0.004246107
29193281,0.004373291,0.004512677,0.004350056,0.004765307,0.004303258,0.004150321,0.004442297,0.004359001,0.004440361,⋯,0.004183550,0.004526751,0.004319073,0.004193111,0.004506164,0.004313829,0.004548497,0.004608670,0.004260511,0.003935823
29198891,0.004415317,0.004507964,0.004374431,0.004584123,0.004228278,0.004445006,0.004435702,0.004352519,0.004250874,⋯,0.004424071,0.004595940,0.004557004,0.004327303,0.004441205,0.004478834,0.004419153,0.004559204,0.004570697,0.004104159
29200651,0.006548581,0.006893473,0.006545926,0.007121203,0.006166059,0.006438496,0.006116101,0.006232703,0.006188036,⋯,0.006082582,0.007001025,0.007275835,0.006432163,0.006205200,0.007384956,0.005877463,0.006247032,0.007528115,0.006683385
29200770,0.003422014,0.003414409,0.003414781,0.003423943,0.003414131,0.003422920,0.003431587,0.003419081,0.003418417,⋯,0.003420812,0.003413979,0.003436244,0.003445578,0.003415430,0.003419253,0.003421099,0.003421002,0.003419907,0.003424083
29201701,0.003580181,0.003555484,0.003618658,0.003592420,0.003646628,0.003620320,0.003528397,0.003564872,0.003559792,⋯,0.003553953,0.003566576,0.003607436,0.003502151,0.003505376,0.003599931,0.003586464,0.003597151,0.003518159,0.003536989


In [132]:
# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

##### Tabla Prediccion

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle

In [134]:
# aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2758031,147.3,5934198,317.0,5934198,317.0
Vcells,249502559,1903.6,788011861,6012.1,2835769564,21635.3


In [137]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "data-mining-analista-jr-2025-a"
PARAM$kaggle$cortes <- seq(1800, 2400, by = 100)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)


for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0(
    "./kaggle/KA",
    PARAM$experimento, "_",
    "s", length(PARAM$FT$semillas), "_",
    envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 
Successfully submitted to Data Mining, Analista Jr 2025A 


In [138]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
notificar("LISTO!!")